In [1]:
import os
import requests
import json
import difflib
from flask import Flask, request, jsonify
from openai import OpenAI

# This assumes you have a config.py file with your tokens.
from config import PAGE_ACCESS_TOKEN, VERIFY_TOKEN, OPENAI_API_KEY

In [2]:
# Initialize the OpenAI client with your API key
client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
app = Flask(__name__)

In [4]:
# --- Load data from files ---
def load_data_from_file(filepath):
    """Loads a JSON file and returns the data."""
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found. Please create it.")
        return {}
    except json.JSONDecodeError:
        print(f"Error: The file '{filepath}' contains invalid JSON. Please check the file format.")
        return {}

In [5]:
# Load all data from JSON files
faq_database = load_data_from_file("faq_english.json")
faq_database_georgian = load_data_from_file("faq_georgian.json")
general_info = load_data_from_file("general_info.json")
default_messages = load_data_from_file("default_messages.json")

In [6]:
# --- Messenger helper ---
def send_message(recipient_id, text):
    """Send message back to a Facebook Messenger user."""
    url = "https://graph.facebook.com/v21.0/me/messages"
    params = {"access_token": PAGE_ACCESS_TOKEN}
    headers = {"Content-Type": "application/json"}
    data = {"recipient": {"id": recipient_id}, "message": {"text": text}}
    r = requests.post(url, params=params, headers=headers, json=data)
    if r.status_code != 200:
        print(f"Error sending message: {r.text}")
        print(f"Facebook API Response: {r.text}")

In [7]:
# --- GPT fallback ---
def get_openai_response(query):
    """
    Sends a query to the OpenAI API and returns the response.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": f"{general_info.get('english', '')}\n\n{general_info.get('georgian', '')}"},
                {"role": "user", "content": query}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return None

In [8]:
# --- Main logic to get an answer ---
def get_answer(query):
    """
    Finds the best answer for a given query using fuzzy matching in both languages.
    Falls back to a general AI response if no match is found.
    """
    # Create a list of all possible questions to match against
    all_questions = list(faq_database.keys()) + list(faq_database_georgian.keys())
    
    # Use fuzzy matching to find the closest match
    matches = difflib.get_close_matches(query.lower(), all_questions, n=1, cutoff=0.6)

    if matches:
        closest_match = matches[0]
        # Check which database the match came from
        if closest_match in faq_database:
            return faq_database[closest_match]
        elif closest_match in faq_database_georgian:
            return faq_database_georgian[closest_match]
    
    # If no close match is found, try to get a response from OpenAI
    openai_response = get_openai_response(query)
    if openai_response:
        return openai_response

    # If the OpenAI response fails, return a default response
    english_default = default_messages.get('english', 'Sorry, I don\'t understand that question.')
    georgian_default = default_messages.get('georgian', 'ბოდიში, ვერ გავიგე ეს შეკითხვა.')
    return f"{english_default}\n\n{georgian_default}"

In [9]:
# --- Root route for testing ---
@app.route("/", methods=["GET"])
def home():
    return "Messenger bot is running!"

In [10]:
# --- Webhook ---
@app.route("/webhook", methods=["GET", "POST"])
def webhook():
    if request.method == "GET":
        # Webhook verification
        token_sent = request.args.get("hub.verify_token")
        return request.args.get("hub.challenge") if token_sent == VERIFY_TOKEN else "Invalid token"
    else:
        # Handle messages
        output = request.get_json()
        for event in output.get("entry", []):
            for messaging_event in event.get("messaging", []):
                sender_id = messaging_event["sender"]["id"]
                if messaging_event.get("message"):
                    user_text = messaging_event["message"].get("text")
                    if user_text:
                        user_text_lower = user_text.lower().strip()
                        response_text = get_answer(user_text)
                        send_message(sender_id, response_text)

        return "ok", 200

In [ ]:
# --- Run server ---
if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Sep/2025 00:36:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:36:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:36:55] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Sep/2025 00:37:29] "GET /webhook?hub.mode=subscribe&hub.challenge=1397990838&hub.verify_token=kdjjskhfjskfnklafmkasa6s45asffsdnfsdf5dsfc HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:38:06] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:38:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:38:35] "GET /webhook?hub.mode=subscribe&hub.challenge=292320272&hub.verify_token=kdjjskhfjskfnklafmkasa6s45asffsdnfsdf5dsfc HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:38:50] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:39:07] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:39:22] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [20/Sep/2025 00:39:27] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [2